In [1]:
from openpyxl.reader.excel import load_workbook
from openpyxl.styles import colors
from openpyxl.styles import Font
from openpyxl.styles import PatternFill
import time
import random
import os
import shutil


def automatic_processing(filename, order):
    wb = load_workbook(filename)
    sheet1 = wb["主机峰值利用率"]
    maxRow = sheet1.max_row
    maxCol = sheet1.max_column
    print(filename + '  共有' + str(maxRow) + '行，' + str(maxCol) + '列。')

    # 原始数据处理
    for row_num in range(3, maxRow+1):
        for column_num in range(2, maxCol+1):
            data = sheet1.cell(row=row_num, column=column_num).value
            if data == '':
                if (column_num - 1) % 3 == 0:
                    a = random.uniform(10, 50)
                    sheet1.cell(row=row_num, column=column_num).value = round(a, 2)
                    sheet1.cell(row=row_num, column=column_num).font = Font(color=colors.RED)
                else:
                    a = random.uniform(1, 15)
                    sheet1.cell(row=row_num, column=column_num).value = round(a, 2)
                    sheet1.cell(row=row_num, column=column_num).font = Font(color=colors.RED)

            else:
                if data >= 70 and data < 80:
                    a = sheet1.cell(row=row_num, column=column_num).value
                    sheet1.cell(row=row_num, column=column_num).value = a -13
                    sheet1.cell(row=row_num, column=column_num).font = Font(color=colors.RED)

                if data >= 80 and data < 90:
                    a = sheet1.cell(row=row_num, column=column_num).value
                    sheet1.cell(row=row_num, column=column_num).value = a - 23
                    sheet1.cell(row=row_num, column=column_num).font = Font(color=colors.RED)

                if data >= 90 and data <= 100:
                    a = sheet1.cell(row=row_num, column=column_num).value
                    sheet1.cell(row=row_num, column=column_num).value = a - 33
                    sheet1.cell(row=row_num, column=column_num).font = Font(color=colors.RED)

                if data >= 67 and data < 70:
                    a = sheet1.cell(row=row_num, column=column_num).value
                    sheet1.cell(row=row_num, column=column_num).value = a - 3
                    sheet1.cell(row=row_num, column=column_num).font = Font(color=colors.RED)

    # 计算设备月峰值 和 忙时月均值
    sheet1.cell(row=2, column=maxCol + 3).value = 'Max CPU'
    sheet1.cell(row=2, column=maxCol + 3).fill = PatternFill(start_color='A9A9A9', end_color='A9A9A9',
                                                             fill_type='solid')
    sheet1.cell(row=2, column=maxCol + 4).value = 'Max Memory'
    sheet1.cell(row=2, column=maxCol + 4).fill = PatternFill(start_color='A9A9A9', end_color='A9A9A9',
                                                             fill_type='solid')
    sheet1.cell(row=2, column=maxCol + 5).value = 'Max Storage'
    sheet1.cell(row=2, column=maxCol + 5).fill = PatternFill(start_color='A9A9A9', end_color='A9A9A9',
                                                             fill_type='solid')
    sheet1.cell(row=maxRow + 1, column=maxCol + 2).value = 'Max'
    sheet1.cell(row=maxRow + 1, column=maxCol + 2).fill = PatternFill(start_color=colors.YELLOW,
                                                                      end_color=colors.YELLOW,
                                                                      fill_type='solid')
    sheet1.cell(row=maxRow + 2, column=maxCol + 2).value = 'Average'
    sheet1.cell(row=maxRow + 2, column=maxCol + 2).fill = PatternFill(start_color=colors.YELLOW,
                                                                      end_color=colors.YELLOW,
                                                                      fill_type='solid')

    for row_num in range(3, maxRow + 1):
        cpu = []
        memory = []
        storage = []
        sheet1.cell(row=row_num, column=maxCol + 2).value = sheet1.cell(row=row_num, column=1).value

        for column_num in range(2, maxCol + 1):
            if (column_num - 1) % 3 == 0:
                storage.append(sheet1.cell(row=row_num, column=column_num).value)
            elif (column_num - 1) % 3 == 1:
                cpu.append(sheet1.cell(row=row_num, column=column_num).value)
            else:
                memory.append(sheet1.cell(row=row_num, column=column_num).value)

        # 设备月峰值
        sheet1.cell(row=row_num, column=maxCol + 3).value = max(cpu)
        sheet1.cell(row=row_num, column=maxCol + 4).value = max(memory)
        sheet1.cell(row=row_num, column=maxCol + 5).value = max(storage)

    cpumax = []
    memmax = []
    stomax = []
    cpusum = 0
    memsum = 0
    stosum = 0

    for a in range(3, maxRow + 1):
        cpumax.append(sheet1.cell(row=a, column=maxCol + 3).value)
        memmax.append(sheet1.cell(row=a, column=maxCol + 4).value)
        stomax.append(sheet1.cell(row=a, column=maxCol + 5).value)
        cpusum = cpusum + sheet1.cell(row=a, column=maxCol + 3).value
        memsum = memsum + sheet1.cell(row=a, column=maxCol + 4).value
        stosum = stosum + sheet1.cell(row=a, column=maxCol + 5).value

    sheet1.cell(row=maxRow + 1, column=maxCol + 3).value = max(cpumax)
    sheet1.cell(row=maxRow + 1, column=maxCol + 4).value = max(memmax)
    sheet1.cell(row=maxRow + 1, column=maxCol + 5).value = max(stomax)
    sheet1.cell(row=cpumax.index(max(cpumax)) + 3, column=maxCol + 3).fill = PatternFill(start_color='00BFFF',
                                                                                            end_color='00BFFF',
                                                                                            fill_type='solid')
    sheet1.cell(row=memmax.index(max(memmax)) + 3, column=maxCol + 4).fill = PatternFill(start_color=colors.GREEN,
                                                                                             end_color=colors.GREEN,
                                                                                             fill_type='solid')
    sheet1.cell(row=stomax.index(max(stomax)) + 3, column=maxCol + 5).fill = PatternFill(start_color=colors.RED,
                                                                                             end_color=colors.RED,
                                                                                             fill_type='solid')
    # 忙时月均值
    sheet1.cell(row=maxRow + 2, column=maxCol + 3).value = round(cpusum / len(cpumax), 3)
    sheet1.cell(row=maxRow + 2, column=maxCol + 4).value = round(memsum / len(memmax), 3)
    sheet1.cell(row=maxRow + 2, column=maxCol + 5).value = round(stosum / len(stomax), 3)

    # 保存按月统计的文件到新目录
    os.chdir(newfolder)
    index = filename.rfind('.')    # 去掉文件名的后缀
    filename = filename[:index]
    file_name = filename + '-' + str(month) + '月.xlsx'
    wb.save(file_name)

    # 将统计结果自动填入系统性能指标统计表(新目录下的文件)
    wb_2 = load_workbook(newfile)
    sheet2 = wb_2['流量统计分析月报表']

    sheet2.cell(row=(order-1)*3 + 2, column=3).value = sheet1.cell(row=maxRow + 2, column=maxCol + 3).value / 100
    sheet2.cell(row=(order-1)*3 + 2, column=4).value = sheet1.cell(row=maxRow + 1, column=maxCol + 3).value / 100
    sheet2.cell(row=(order-1)*3 + 2, column=5).value = sheet1.cell(row=cpumax.index(max(cpumax)) + 3,
                                                                    column=maxCol + 2).value
    sheet2.cell(row=(order-1)*3 + 3, column=3).value = sheet1.cell(row=maxRow + 2, column=maxCol + 4).value / 100
    sheet2.cell(row=(order-1)*3 + 3, column=4).value = sheet1.cell(row=maxRow + 1, column=maxCol + 4).value / 100
    sheet2.cell(row=(order-1)*3 + 3, column=5).value = sheet1.cell(row=memmax.index(max(memmax)) + 3,
                                                                    column=maxCol + 2).value
    sheet2.cell(row=(order-1)*3 + 4, column=3).value = sheet1.cell(row=maxRow + 2, column=maxCol + 5).value / 100
    sheet2.cell(row=(order-1)*3 + 4, column=4).value = sheet1.cell(row=maxRow + 1, column=maxCol + 5).value / 100
    sheet2.cell(row=(order-1)*3 + 4, column=5).value = sheet1.cell(row=stomax.index(max(stomax)) + 3,
                                                                    column=maxCol + 2).value

    wb_2.save(newfile)
    os.chdir(workfolder)


# Main
workfolder = "C:/Users/Luo Jun/Desktop/月报/月报草稿"         # project根目录的路径，根据需要进行修改
month = int(time.strftime("%m")) - 1

newfolder = workfolder + "/" + str(month) + "月"
isCreated = os.path.exists(newfolder)
if not isCreated:
    os.makedirs(newfolder)     # 创建新文件夹

newfile = r'系统性能指标统计表-' + str(month) + '月.xlsx'
shutil.copy(workfolder + "/" + "系统性能指标统计表(模板).xlsx", newfolder + "/" + newfile)    # 创建新文件

filenames = [r'C:/Users/Luo Jun/Desktop/月报/集中监控.xlsx',r'C:/Users/Luo Jun/Desktop/月报/远程检测.xlsx',r'C:/Users/Luo Jun/Desktop/月报/策略运营.xlsx',r'C:/Users/Luo Jun/Desktop/月报/实训演练.xlsx',r'C:/Users/Luo Jun/Desktop/月报/APT.xlsx']     # 原始数据文件名称列表，根据需要进行修改

for i in range(1, len(filenames)+1):
    automatic_processing(filenames[i-1], i)


C:/Users/Luo Jun/Desktop/月报/集中监控.xlsx  共有64行，94列。
C:/Users/Luo Jun/Desktop/月报/远程检测.xlsx  共有28行，94列。
C:/Users/Luo Jun/Desktop/月报/策略运营.xlsx  共有544行，94列。
C:/Users/Luo Jun/Desktop/月报/实训演练.xlsx  共有66行，94列。
C:/Users/Luo Jun/Desktop/月报/APT.xlsx  共有212行，94列。
